# Load Data

In [1]:
import tensorflow as tf

# Load the Fashion MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

## Data Prep

In [1]:
# Subset your data the same 3 digit classes you used in the last assignments.

# Again, You can totally copy any code over and modify it.
# You can use chatGPT, copilot, google, or other AI or online resources.
# Use each other, the goal is to complete the objectives and maybe learn something new, not to struggle to make up code on your own.
# We have prior notebooks, chatGPT, the internet, and each other for the rapids projects, so use them here if they will help!

# Extract the 3 classes from the training data

'Your code here'

# Extract the 3 classes from the test data

'Your code here'

'Your code here'

## Plot Data

In [2]:
## Plot 6 to 10 images from your data

'Your code here'


'Your code here'

# Transfer Learning

## Load and Modify Model

In [3]:
from torchvision.models import mobilenet_v2
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Load and modify a pre-trained mobilenet_v2 model for your dataset
# Make sure to change the output layer for the number of classes you are using

'Your code here'

'Your code here'

## Modify Data to fit with Model

In [4]:
import torchvision.transforms as transforms
import torch

# Pre-trained models may not match the structure of our data.
# It's best to modify the model to handle the data, but we may need to adjust the images as well.
# Resize the images to 32x32 for the model

'Your code here'


'Your code here'

## Fit Model on our Data

In [5]:
# Convert your data to tensors and create a DataLoader

'Your code here'

# Update your model with our training data
# Report some notion of accuracy, loss or both as it builds to see how well your model is doing.

'Your code here'

'Your code here'

## Confusion Matrix

In [6]:
# Compute and plot a confusion matrix.

'Your code here'

# Where are misclassifications occurring?
# How does your test accuracy compare to that of the Random Forest and CNN made previously?
# It's very possible that it won't be better than the CNN.
# This hopefully illustrates the importance of choosing the right kind of model to transfer learn from.

'Your code here'

## Plot Misclassifications

In [7]:
# Plot some of the misclassifications.

'Your code here'

# Can you see why the model made the misclassifications it did?

'Your code here'